In [ ]:
%spark.conf
influxdb.url https://css21.teco.edu
influxdb.token d5oSFVlZ-7TuaJgq4XYosp-6E5Bh_6MsJAit7GbcshHdUh7mKy5v-pFGfH4DGg775t_FwpK7pTsKDItRiM9nJQ\u003d\u003d
influxdb.org css21


In [1]:
%influxdb(saveAs\u003d"events")
result \u003d from(bucket: "extended_labels")
      |\u003e range(start: -30d, stop: now())
      |\u003e filter(fn: (r) \u003d\u003e r.label !\u003d "testing")
      |\u003e pivot(rowKey: ["_time", "label", "subject"], columnKey: ["_field", "_measurement"], valueColumn: "_value")
     
 yield result

In [2]:
from io import StringIO
import pandas as pd


df =  df.drop(columns\u003d["_start", "result", "table", "_stop"])
df  spark.createDataFrame(df)
z.show(df)

In [3]:
from pyspark.sql.functions import col,  window, mean, sum as _sum, max as _max, min as _min, var_samp, to_timestamp
from pyspark.sql.window import Window


df = df.withColumn("_time", to_timestamp("_time"))
w = window("_time", "1 seconds")

aggregate = ["alpha_devicemotion", "beta_devicemotion", "gamma_devicemotion", "x_devicemotion", "y_devicemotion", "z_devicemotion", "alpha_deviceorientation", "beta_deviceorientation", "gamma_deviceorientation"] 
funs = [mean, _sum, _max, var_samp, _min]

exprs = [f(col(c)) for f in funs for c in aggregate]

grouped = df.fillna(0).groupBy([w, "subject", "label"]).agg(*exprs)
grouped = grouped.drop("subject", "window")
grouped

z.show(grouped)

In [4]:
%spark.ipyspark
from pyspark.sql.types import StringType, DoubleType

num_cols = [f.name for f in grouped.schema.fields if isinstance(f.dataType, DoubleType)]

num_cols

In [5]:
%spark.ipyspark

from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer,  VectorIndexer, VectorAssembler

#indexer \u003d StringIndexer(inputCol\u003d"label", outputCol\u003d"class")
#classifier \u003d DecisionTreeClassifier(labelCol\u003d"class")
#pipeline \u003d Pipeline(stages \u003d [indexer, classifier])
#pipeline.fit(grouped)

# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
labelIndexer \u003d StringIndexer(inputCol\u003d"label", outputCol\u003d"indexedLabel").fit(grouped)
# Automatically identify categorical features, and index them.
# We specify maxCategories so features with \u003e 4 distinct values are treated as continuous.
featureAssembler \u003d VectorAssembler(inputCols \u003d num_cols, outputCol \u003d \u0027features\u0027)

# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) \u003d grouped.randomSplit([0.7, 0.3])

# Train a DecisionTree model.
dt \u003d DecisionTreeClassifier(labelCol\u003d"indexedLabel", featuresCol\u003d"features")

# Chain indexers and tree in a Pipeline
pipeline \u003d Pipeline(stages\u003d[labelIndexer, featureAssembler, dt])

# Train model.  This also runs the indexers.
model \u003d pipeline.fit(trainingData)




In [6]:
%spark.ipyspark

from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Make predictions.
predictions \u003d model.transform(testData)

# Select example rows to display.
predictions.select("prediction", "indexedLabel", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator \u003d MulticlassClassificationEvaluator(
    labelCol\u003d"indexedLabel", predictionCol\u003d"prediction", metricName\u003d"accuracy")
accuracy \u003d evaluator.evaluate(predictions)
print("Test Error \u003d %g " % (1.0 - accuracy))


In [7]:
%spark.pyspark

model.write().save("/notebook/ks_context")

In [8]:
%spark.ipyspark
